In [119]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [120]:
pip install tflearn

Note: you may need to restart the kernel to use updated packages.


In [121]:
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

In [122]:
# import our chat-bot intents file
with open(r'C:\Users\manigandan\Desktop\hackathon rit\intents.json') as json_data:
    intents = json.load(json_data)

In [123]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['hi', 'How are you', 'Hello', 'Hey', 'Hai'],
   'responses': ['Hi,How are you?',
    'Hey, How are you?',
    'Hello, How are you?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'see you later', 'good bye'],
   'responses': ['BYE', 'see you later', 'Take Care']},
  {'tag': 'thanks',
   'patterns': ['thankyou', 'thanks', 'thats helpful'],
   'responses': ['Youre Welcome', 'My pleasure', 'Thanks']},
  {'tag': 'positive_health',
   'patterns': ['feeling great', 'better', 'good', 'fine'],
   'responses': ['Okay', 'Im your Quarantine Support']},
  {'tag': 'negative_health',
   'patterns': ['Feeling bad',
    'uncomfortable',
    'uneasy',
    'feverish',
    'tired',
    'dull',
    'dizzy'],
   'responses': ["Don't Worry! I will take care of you",
    "I'm here to entertain you",
    "Don't worry, you will be alright"]},
  {'tag': 'bored',
   'patterns': ['im bored', 'bored', 'bore', 'im feeling bored'],
   'responses'

In [124]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [125]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

34 documents
8 classes ['bored', 'goodbye', 'greeting', 'jokes', 'negative_health', 'positive_health', 'riddle', 'thanks']
37 unique stemmed words ['ar', 'bad', 'bet', 'bor', 'bye', 'comedy', 'dizzy', 'dul', 'feel', 'fev', 'fin', 'funny', 'gam', 'good', 'gre', 'hai', 'hello', 'help', 'hey', 'hi', 'how', 'im', 'jok', 'lat', 'me', 'riddl', 'see', 'som', 'tel', 'thank', 'thankyou', 'that', 'tir', 'uncomfort', 'uneasy', 'word', 'you']


In [126]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [127]:
# resetting underlying graph data
tf.reset_default_graph()

# Building neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 4999  | total loss: 0.22790 | time: 0.017s
| Adam | epoch: 1000 | loss: 0.22790 - acc: 0.9877 -- iter: 32/34
Training Step: 5000  | total loss: 0.21111 | time: 0.020s
| Adam | epoch: 1000 | loss: 0.21111 - acc: 0.9889 -- iter: 34/34
--
INFO:tensorflow:C:\Users\manigandan\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [128]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [129]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [130]:
with open(r'C:\Users\manigandan\Desktop\hackathon rit\intents.json') as json_data:
    intents = json.load(json_data)

In [131]:


# load the saved model
model.load('./model.tflearn')



INFO:tensorflow:Restoring parameters from C:\Users\manigandan\model.tflearn


In [132]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [133]:

ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return random.choice(i['responses'])

            results.pop(0)

In [134]:
import random
import tkinter
from tkinter import *
def rock_paper_scissor():
    #Creating GUI with tkinter
   
    def play():
        user_choice = EntryBox.get("1.0",'end-1c').strip()
        EntryBox.delete("0.0",END)
        if user_choice != '':
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, "You: " +user_choice + '\n\n')
            ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
            select = ['rock','paper','scissor']
            bot_choice = random.choice(select)
            ChatLog.insert(END, "Bot: " +bot_choice + '\n\n')
            if(user_choice.lower()=='rock' and bot_choice.lower()=='rock'):
                ChatLog.insert(END, "MATCH DRAWS \n \n")
                ChatLog.insert(END, "-------------------------------------- \n \n")
            if(user_choice.lower()=='rock' and bot_choice.lower()=='paper'):
                ChatLog.insert(END, "BOT WINS \n \n")
                ChatLog.insert(END, "-------------------------------------- \n \n")
            if(user_choice.lower()=='rock' and bot_choice.lower()=='scissor'):
                ChatLog.insert(END, "YOU WINS \n \n")
                ChatLog.insert(END, "-------------------------------------- \n \n")
            if(user_choice.lower()=='paper' and bot_choice.lower()=='rock'):
                ChatLog.insert(END, "YOU WINS \n \n")
                ChatLog.insert(END, "-------------------------------------- \n \n")
            if(user_choice.lower()=='paper' and bot_choice.lower()=='paper'):
                ChatLog.insert(END, "MATCH DRAWS \n \n")
                ChatLog.insert(END, "-------------------------------------- \n \n")
            if(user_choice.lower()=='paper' and bot_choice.lower()=='scissor'):
                ChatLog.insert(END, "BOT WINS \n \n")
                ChatLog.insert(END, "-------------------------------------- \n \n")
            if(user_choice.lower()=='scissor' and bot_choice.lower()=='rock'):
                ChatLog.insert(END, "BOT WINS \n \n")
                ChatLog.insert(END, "-------------------------------------- \n \n")
            if(user_choice.lower()=='scissor' and bot_choice.lower()=='paper'):
                ChatLog.insert(END, "YOU WINS \n \n")
                ChatLog.insert(END, "-------------------------------------- \n \n")    
            if(user_choice.lower()=='scissor' and bot_choice.lower()=='scissor'):
                ChatLog.insert(END, "MATCH DRAWS \n \n")
                ChatLog.insert(END, "-------------------------------------- \n \n")
            ChatLog.insert(END, "BOT: choose any one \n 1.ROCK \n 2. PAPER \n 3.SCISSOR \n ")
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)



    base = Tk()
    base.title("GAME")
    base.geometry("400x500")
    base.resizable(width=FALSE, height=FALSE)

    #Create Chat window
    ChatLog = Text(base, bd=0, bg="#FFCCCC", height="8", width="50", font="Arial",)
    ChatLog.insert(END, "BOT: choose any one \n 1.ROCK \n 2. PAPER \n 3.SCISSOR \n "  )
    ChatLog.config(state=DISABLED)
    #Bind scrollbar to Chat window
    scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
    ChatLog['yscrollcommand'] = scrollbar.set
    #Create Button to send message
    SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= play)
    #Create the box to enter message
    EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
    #EntryBox.bind("<Return>", send)
    #Place all components on the screen
    scrollbar.place(x=376,y=6, height=386)
    ChatLog.place(x=6,y=6, height=386, width=370)
    EntryBox.place(x=128, y=401, height=90, width=265)
    SendButton.place(x=6, y=401, height=90)
    base.mainloop()

In [140]:
#Creating GUI with tkinter
import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        res = response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
base = Tk()
base.title("CHATBOT")
base.geometry("400x500")

base.resizable(width=FALSE, height=FALSE)
#Create Chat window
ChatLog = Text(base, bd=0, bg="#CCFFFF", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
#Create Button to send message
sendButton = Button(base, font=("Verdana",12,'bold'), text="send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )
playButton = Button(base, font=("Verdana",12,'bold'), text="PLAY", width="6", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= rock_paper_scissor )
#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
sendButton.place(x=6, y=401, height=90)
playButton.place(x=300,y=10, height=40)
base.mainloop()

In [117]:
#sentimental analysis
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import names

def word_feats(words):
    return dict([(word, True) for word in words])
 
positive_vocab = [ 'awesome', 'outstanding', 'fantastic', 'terrific', 'good', 'nice', 'great', ':)' ]
negative_vocab = [ 'bad', 'terrible','useless', 'hate', ':(','not' ]


positive_features = [(word_feats(pos), 'pos') for pos in positive_vocab]
negative_features = [(word_feats(neg), 'neg') for neg in negative_vocab]


train_set = negative_features + positive_features 

classifier = NaiveBayesClassifier.train(train_set) 

# Predict
neg = 0
pos = 0
sentence = input()
sentence = sentence.lower()
words = sentence.split(' ')
for word in words:
    classResult = classifier.classify( word_feats(word))
    if classResult == 'neg':
        neg = neg + 1
    if classResult == 'pos':
        pos = pos + 1

print('Positive: ' + str(float(pos)/len(words)))
print('Negative: ' + str(float(neg)/len(words)))



Positive: 1.0
Negative: 0.0
